In [ ]:
from MFEA_lib.model import AbstractModel, SMP_MFEA, MFEA_base, SMP_DE
from MFEA_lib.model.utils import *
from MFEA_lib.model.utils import TuningModel
import pandas as pd
import numpy as np
import scipy.io
from MFEA_lib.tasks.Benchmark.Funcs import * 
from MFEA_lib.operators.Crossover import *
from MFEA_lib.operators.Mutation import *
from MFEA_lib.operators.Selection import *


### Draft

In [ ]:
a = [4,5,6] 
for idx, d in enumerate(a[1:], start= 1):
    print(idx)
    print(d)

### Tuning Model


In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
name_benchmarks = []
for id in range(1, 10):
    # print(id) 
    b, i = WCCI22_benchmark.get_complex_benchmark(4)
    name_benchmarks.append("wcci_2tasks_" + str(id))
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)
name_benchmarks.append("CEC_10_")

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 3): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

In [ ]:
model = TuningModel(
    model_name= SMP_MFEA, 
    list_parameter= [
        ('crossover', {
            'gamma': [0.4, 0.6]
        }), 
        ('lr', [0.04, 0.05]),
        ('crossover', {
            'alpha': [5, 7]
        }), 
    ],
)
model.compile(
    ls_benchmark = ls_benchmark, 
    benchmark_weights = (np.ones(shape= (3,)) / 3).tolist(), 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,

    # tasks = None,
    # IndClass= None,
    # tasks = CEC17_benchmark.get_10tasks_benchmark(),
    crossover= newSBX(nc = 2, gamma= 0.4, alpha = 6),
    # crossover= SBX_Crossover(nc = 2),
    mutation = GaussMutation(scale = 0.01),
    selection= ElitismSelection(random_percent= 0.),
    # search= SHADE(len_mem= 30, p_ontop= 0.1), 
    attr_tasks = ['crossover', 'mutation']
)

fit, com, result = model.run(
    replace_folder=True,
    min_value = 1e-6,
    
    nb_generations= 5, 
    nb_inds_each_task= 100, 
    nb_inds_min= 100,
    lr = 0.05,  
    p_const_intra= 0, 
    p_mutate= 0.1, 
    prob_search = 0.,
    nb_epochs_stop= 1000, 
    swap_po= False,
    evaluate_initial_skillFactor= True
)

### Show and compare result

In [ ]:
for id in range(1, 11): 
    SMP = loadModel("./RESULTS/50tasks/SMP_NEWSBX_TUN_50/Gecco_id_"+ str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    SA = loadModel("./RESULTS/50tasks/SA/SA_Benchmark_"+str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    LSA_21 = loadModel("./RESULTS/50tasks/LSA_2021/LSA_2021_MTOMSO_Benchmark_"+str(id) + ".mso", GECCO20_benchmark_50tasks.get_items(id)[0])
    com = CompareModel([SMP, SA, LSA_21])
    print(com.detail_compare_result(min_value= 1e-6, round = 7))

In [ ]:
MFEA_model = loadModel('./RESULTS/MFEA_cec17.mso')
EBS_GA_model = loadModel('./RESULTS/EBS_GA_cec17.mso')
MaTDE_model = loadModel('./RESULTS/MaTDE_cec17.mso')
MaTGA_model = loadModel('./RESULTS/MaTGA_cec17.mso')
MTEA_AD_model = loadModel('./RESULTS/MTEA_AD_cec17.mso')
SA_model = loadModel('./RESULTS/SA_cec17.mso')
LSA_model = loadModel('./RESULTS/LSA_cec17.mso')
# SMP_NEWSBX = loadModel("./RESULTS/SMP_NEWSBX.mso", ls_tasks= CEC17_benchmark.get_10tasks_benchmark())


In [ ]:
compare = CompareModel(
    models = [MFEA_model, EBS_GA_model, MaTDE_model, MaTGA_model, MTEA_AD_model, SA_model, LSA_model],
    label= [...] * 7 ,
)
# compare.render(    
#     shape = (2, 5),
#     min_cost= 0,
#     step= 100,
#     yscale= 'log'
# )

In [ ]:
compare.detail_compare_result()

In [ ]:
compare.summarizing_compare_result(path=None , idx_main_algo= 2, nb_task= 2, ls_benchmark= [CEC17_benchmark.get_2tasks_benchmark(id) for id in range(1,10)])

In [ ]:
model = loadModel("./RESULTS/50tasks/SMP_NEWSBX/lan2_2s_newsbx_50task_1.mso")

In [ ]:
compare.summarizing_compare_result(path="./RESULTS/complex_2tasks/", idx_main_algo= 0, nb_task= 2, ls_benchmark= [WCCI22_benchmark.get_complex_benchmark(id)[0] for id in range(1,10)],min_value= 1e-6, combine= True)

In [ ]:
compare.summarizing_compare_result(path="./RESULTS/50tasks/", idx_main_algo= 7, nb_task= 50, ls_benchmark= [GECCO20_benchmark_50tasks.get_items(id) for id in range(1,11)],min_value= 1e-6, combine= False)

### Multi Time Model

In [ ]:
tasks, IndClass= CEC17_benchmark.get_10tasks_benchmark()

smpModel = MultiTimeModel(model= SMP_MFEA)
smpModel.compile(
    tasks= tasks,
    IndClass = IndClass,
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 1000, nb_inds_each_task= 100, nb_inds_min= 100,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

In [ ]:
smpModel.print_result(['history_cost'])

In [ ]:
ls_benchmark = [] 
ls_IndClass = [] 
for id in range(1, 3): 
    b, i = CEC17_benchmark.get_2tasks_benchmark(id) 
    ls_benchmark.append(b) 
    ls_IndClass.append(i) 

b, i = CEC17_benchmark.get_10tasks_benchmark() 
ls_benchmark.append(b)
ls_IndClass.append(i)

smpModel = MultiBenchmark(
    ls_benchmark = ls_benchmark, 
    name_benchmark = [str(i) for i in range(1,4)],
    ls_IndClass = ls_IndClass,
    model= SMP_MFEA
    )

smpModel.compile(
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    attr_tasks = ['crossover', 'mutation'],
    
)
smpModel.fit(
    nb_generations= 10, nb_inds_each_task= 100, nb_inds_min= 100,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.0,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)

In [ ]:
smpModel.print_result(['history_cost'])

### SMP DE

In [ ]:
tasks, IndClass= CEC17_benchmark.get_10tasks_benchmark()

smpModel = MultiTimeModel(model= SMP_DE)
smpModel.compile(
    tasks= tasks,
    IndClass = IndClass,
    crossover= newSBX(nc = 2, gamma= 0.6, alpha = 7),
    mutation= GaussMutation(scale= 0.1),
    # mutation = GMMScale(alpha = 3, lenMem= 20, default_scale= 0.5),
    selection= ElitismSelection(random_percent= 0.),
    search = SHADE(len_mem=30, p_ontop= 0.1),
    attr_tasks = ['crossover', 'mutation', 'search'],
    
)
smpModel.fit(
    nb_generations= 10, nb_inds_each_task= 100, nb_inds_min= 100,
    lr = 0.1, p_const_intra= 0.0, p_mutate= 0.1,prob_search = 0.2,
    nb_epochs_stop= 50, 
    evaluate_initial_skillFactor= True
)
smpModel.run(
    nb_run= 2,
    save_path= './RESULTS/result/smpMFEA_cec17.mso'
)